In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import rcParams
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

In [2]:
# allow output to span multiple output lines in the console
pd.set_option('display.max_columns', 500)

In [3]:
# switch to seaborn default stylistic parameters
# see the useful https://seaborn.pydata.org/tutorial/aesthetics.html
sns.set()
sns.set_context('paper') # 'talk' for slightly larger

# change default plot size
rcParams['figure.figsize'] = 9,7

In [4]:
# column names, without the chess moves column
misc_columnNames = ['PNG_File_Pos - DELETE ME', 'Date of Game', 'Game Result', 'W-ELO', 'B-ELO', 
                    'Num Moves', 'miscDate - DELETE ME', 'result - DELETE ME', 'wELO - DELETE ME', 'bELO - DELETE ME', 
                    'event date - DELETE ME', 'setup - DELETE ME', 'fen - DELETE ME', 'flag - DELETE ME', 'oyrange - DELETE ME', 
                    'bad len - DELETE ME']


In [5]:
# read all data except chess moves,
misc_chess_data = pd.read_csv('https://raw.githubusercontent.com/abecsumb/DataScienceProject/main/Chess_Data.txt', comment = '#', infer_datetime_format = True, header = None, sep = ' ', on_bad_lines = 'skip')
misc_chess_data.drop(misc_chess_data.columns[16], axis = 1, inplace = True)
misc_chess_data.columns = misc_columnNames


In [6]:
# Isolate game moves from everything else.
game_moves = pd.read_csv('https://raw.githubusercontent.com/abecsumb/DataScienceProject/main/Chess_Data.txt', sep = '###', on_bad_lines = 'skip', header = None)


C:\Users\Abrah\AppData\Local\Temp\ipykernel_8904\1637265906.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  game_moves = pd.read_csv('https://raw.githubusercontent.com/abecsumb/DataScienceProject/main/Chess_Data.txt', sep = '###', on_bad_lines = 'skip', header = None)


In [7]:
# drop first column of game moves (this is the misc chess data)
game_moves.drop(game_moves.columns[0], axis = 1, inplace = True)


In [8]:
# split game moves df into columns for each move. 
game_moves = game_moves.iloc[:, 0].str.lstrip()
game_moves = game_moves.iloc[:].str.split(pat = ' ', expand = True)


In [9]:
# merge misc data and game moves into one df, and drop all unnecessary columns
chess_data = pd.concat([misc_chess_data, game_moves], axis = 1)
chess_data.drop(labels = ['PNG_File_Pos - DELETE ME', 'miscDate - DELETE ME', 'result - DELETE ME', 
               'wELO - DELETE ME', 'bELO - DELETE ME', 'event date - DELETE ME', 
               'setup - DELETE ME', 'fen - DELETE ME', 'flag - DELETE ME', 'oyrange - DELETE ME', 'bad len - DELETE ME'], axis = 1, inplace = True)


In [10]:
chess_data

,Date of Game,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313
0,2000.03.14,1-0,2851,None,67,W1.d4,B1.d5,W2.c4,B2.e6,W3.Nc3,B3.Nf6,W4.cxd5,B4.exd5,W5.Bg5,B5.Be7,W6.e3,B6.Ne4,W7.Bxe7,B7.Nxc3,W8.Bxd8,B8.Nxd1,W9.Bxc7,B9.Nxb2,W10.Rb1,B10.Nc4,W11.Bxc4,B11.dxc4,W12.Ne2,B12.O-O,W13.Nc3,B13.b6,W14.d5,B14.Na6,W15.Bd6,B15.Rd8,W16.Ba3,B16.Bb7,W17.e4,B17.f6,W18.Ke2,B18.Nc7,W19.Rhd1,B19.Ba6,W20.Ke3,B20.Kf7,W21.g4,B21.g5,W22.h4,B22.h6,W23.Rh1,B23.Re8,W24.f3,B24.Bb7,W25.hxg5,B25.fxg5,W26.d6,B26.Nd5+,W27.Nxd5,B27.Bxd5,W28.Rxh6,B28.c3,W29.d7,B29.Re6,W30.Rh7+,B30.Kg8,W31.Rbh1,B31.Bc6,W32.Rh8+,B32.Kf7,W33.Rxa8,B33.Bxd7,W34.Rh7+,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,2000.03.14,1-0,2851,None,53,W1.e4,B1.d5,W2.exd5,B2.Qxd5,W3.Nc3,B3.Qa5,W4.d4,B4.Nf6,W5.Nf3,B5.c6,W6.Ne5,B6.Bf5,W7.g4,B7.Be4,W8.f3,B8.Bd5,W9.a3,B9.Nbd7,W10.Be3,B10.Nxe5,W11.dxe5,B11.Nxg4,W12.Bd4,B12.e6,W13.b4,B13.Qd8,W14.Nxd5,B14.Qxd5,W15.c4,B15.Ne3,W16.cxd5,B16.Nxd1,W17.dxc6,B17.bxc6,W18.Rxd1,B18.Be7,W19.Ba6,B19.O-O,W20.Ke2,B20.Rab8,W21.Rc1,B21.Rfd8,W22.Rhd1,B22.c5,W23.Bxc5,B23.Rxd1,W24.Rxd1,B24.Bxc5,W25.bxc5,B25.g6,W26.c6,B26.Rb2+,W27.Rd2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,No